## 대구광역시 공동주택 정보 (공공데이터포털 - 국토교통부)
+ 작성: 임경호

In [1]:
import requests
import xmltodict
import pandas as pd

# 일반 인증키(Encoding)	
api_key = 'IVgu%2FZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60%2Fs2EYAMUsyyzTqwVOnjoIhg%3D%3D'

#### 1. 대구광역시 공동주택 목록 가져오기

In [ ]:
service_url = 'http://apis.data.go.kr/1613000/AptListService2'
service_name = 'getSidoAptList'
sido_code = '27'    # 대구광역시
rows = '1500'
url = f'{service_url}/{service_name}?sidoCode={sido_code}&ServiceKey={api_key}&numOfRows={rows}'

response = requests.get(url,verify=False)

if response.status_code == 200:     # URL GET '200 정상'
    print("URL GET Successful")
    contents = response.text                # XML 데이터
    # xmltodict 모듈을 사용해서 xml 데이터를 파이썬의 OrderedDict 타입으로 변환
    dict_data = xmltodict.parse(contents)
    item = dict_data['response']['body']['items']['item']
    df_apt = pd.json_normalize(item)
else:
    print("URL GET Error")
    
df_apt_codes = df_apt['kaptCode']

In [3]:
total_code_cnts = df_apt['as2'].size

In [4]:
region_list = df_apt['as2'].unique()
dict_region = {}
for region in region_list:
    dict_region[region] = df_apt.query('as2 == @region')['kaptCode'].to_list()

In [5]:
for key, value in dict_region.items():
    print(key, end=" ")

달서구 동구 남구 중구 수성구 북구 서구 달성군 군위군 

#### 2. 대구광역시 공동주택 기본정보 가져오기
+ 일일 트래픽 제한: 10,000건

In [6]:
# 각 지역별로 데이터 수집 결과 초기화
dict_region_results = {}
for key in dict_region.keys():
    dict_region_results[key] = 0
print(dict_region_results)

{'달서구': 0, '동구': 0, '남구': 0, '중구': 0, '수성구': 0, '북구': 0, '서구': 0, '달성군': 0, '군위군': 0}


In [7]:
from tqdm.notebook import tqdm
import time

import warnings
warnings.filterwarnings(action='ignore')

service_url = 'http://apis.data.go.kr/1613000/AptBasisInfoService1'
service_name = 'getAphusBassInfo'   # 공동주택 기본 정보제공 서비스

dict_region_data = {}

for region, apt_code_list in dict_region.items():   
    df_region_apt_info = pd.DataFrame()
    # 최대 5회 시도
    num_retry = 7
    for retry in range(num_retry):
        if dict_region_results[region] == 0: 
            try:
                print(f"{region} 지역 아파트 정보를 가져옵니다. 대상 목록 {len(apt_code_list)}, trying {retry}")
                for apt_code in tqdm(apt_code_list):    
                    url = f'{service_url}/{service_name}?kaptCode={apt_code}&ServiceKey={api_key}'
                    response = requests.get(url,verify=False)
                    if response.status_code == 200:     # URL GET '200 정상'
                        contents = response.text                # XML 데이터
                        # xmltodict 모듈을 사용해서 xml 데이터를 파이썬의 OrderedDict 타입으로 변환
                        dict_data = xmltodict.parse(contents)
                        item = dict_data['response']['body']['item']
                        df_apt_info = pd.json_normalize(item)
                        if not df_apt_info.empty:
                            df_region_apt_info = pd.concat([df_region_apt_info, df_apt_info], ignore_index=True)
                if not df_region_apt_info.empty:            
                    dict_region_data[region] = df_region_apt_info
                    dict_region_results[region] = 1    # 성공
                    break
                else:
                    dict_region_results[region] = 0    # 실패
            except Exception:
                continue     
    time.sleep(5)

달서구 지역 아파트 정보를 가져옵니다. 대상 목록 219, trying 0


  0%|          | 0/219 [00:00<?, ?it/s]

동구 지역 아파트 정보를 가져옵니다. 대상 목록 144, trying 0


  0%|          | 0/144 [00:00<?, ?it/s]

남구 지역 아파트 정보를 가져옵니다. 대상 목록 36, trying 0


...

  0%|          | 0/19 [00:00<?, ?it/s]

달성군 지역 아파트 정보를 가져옵니다. 대상 목록 128, trying 0


  0%|          | 0/128 [00:00<?, ?it/s]

군위군 지역 아파트 정보를 가져옵니다. 대상 목록 1, trying 0


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
# 각 지역별로 데이터 수집 결과 확인: 성공(1), 실패(0)
print("각 지역별 데이터 수집 결과")
for key in dict_region_results:
    print(f'{key}\t{dict_region_results[key]}')

각 지역별 데이터 수집 결과
달서구	1
동구	1
남구	1
중구	1
수성구	1
북구	1
서구	1
달성군	1
군위군	1


In [9]:
# 데이터를 가져오지 못한 지역
[key for key, value in dict_region_results.items() if value == 0]
if len(key) == 0:
    print("모든 지역의 데이터를 수집하였습니다.")
else:
    print("데이터를 수집하지 못한 지역이 있습니다.")

데이터를 수집하지 못한 지역이 있습니다.


In [12]:
df_apt_codes.size

1002

In [10]:
total_counts = 0
for region in dict_region.keys():
    if dict_region_results[region] == 1:
        cnt = dict_region_data[region]['kaptCode'].size
        print(f"{region}\t: {cnt}")
        total_counts += cnt
    else:
        print(f"{region}\t: 0")
        continue
print(f"총 개수\t: {total_counts}")

달서구	: 219
동구	: 144
남구	: 36
중구	: 38
수성구	: 221
북구	: 196
서구	: 19
달성군	: 128
군위군	: 1
총 개수	: 1002


### 항목 내역 (활용할 만한 사항들만 발췌)
+ 단지코드	kaptCode	9
+ 단지명	kaptName	50
+ 법정동주소	kaptAddr	200
+ 분양형태  codeSaleNm	50
+ 난방방식	codeHeatNm	50
+ 연면적  kaptTarea 22  
+ 동수	kaptDongCnt	100
+ 세대수	kaptdaCnt	22	
+ 단지분류	codeAptNm	50
+ 호수	hoCnt	5
+ 관리방식  codeMgrNm   50
+ 복도유형  codeHallNm  50
+ 사용승인일	kaptUsedate	8
+ 전용면적별 세대현황	kaptMparea_60	22  : 60㎡ 이하
+ 전용면적별 세대현황	kaptMparea_85	22  : 60㎡ ~ 85㎡ 이하
+ 전용면적별 세대현황	kaptMparea_135	22  : 85㎡ ~ 135㎡ 이하
+ 전용면적별 세대현황	kaptMparea_136	22  : 135㎡ 초과
+ 대장 전용면적합계	privArea	22  : 단지 전용면적합(㎡)
+ 법정동코드	bjdCode	10

In [ ]:
# Null Value 찾기
# df_save_data.isnull().sum()

#### 3. 데이터 저장

In [13]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

# 한꺼번에 저장할 경우 Error 발생. 구군 지역별로 저장
for region in dict_region_data.keys():
    df_region_data = dict_region_data[region]
    df_save_data = df_region_data[['kaptCode', 'kaptName', 'kaptAddr', 'codeSaleNm', 'codeHeatNm', 'kaptTarea', \
                                   'kaptDongCnt', 'kaptdaCnt', 'codeAptNm', 'hoCnt', 'codeMgrNm', 'codeHallNm',  \
                                   'kaptUsedate', 'kaptMparea_60', 'kaptMparea_85', 'kaptMparea_135', 'kaptMparea_136', \
                                   'privArea', 'bjdCode']]
    # NaN 값을 공백으로 변환 (Table에 저장하기 위함)
    df_save_data['codeSaleNm'] = df_save_data['codeSaleNm'].fillna('')
    df_save_data['codeHeatNm'] = df_save_data['codeHeatNm'].fillna('')
    df_save_data['codeAptNm'] = df_save_data['codeAptNm'].fillna('')
    df_save_data['codeMgrNm'] = df_save_data['codeMgrNm'].fillna('')
    df_save_data['codeHallNm'] = df_save_data['codeHallNm'].fillna('')
    df_save_data['kaptUsedate'] = df_save_data['kaptUsedate'].fillna('')

    # NaN 값을 0로 변환 (Table에 저장하기 위함)
    df_save_data['kaptDongCnt'] = df_save_data['kaptDongCnt'].fillna(0)
    df_save_data['hoCnt'] = df_save_data['hoCnt'].fillna(0)
    
    for row in df_save_data.itertuples():
        sql = "insert into molit_apt_daegu (kaptCode, kaptName, kaptAddr, codeSaleNm, codeHeatNm, kaptTarea, \
                                            kaptDongCnt, kaptdaCnt, codeAptNm, hoCnt, codeMgrNm, codeHallNm,  \
                                            kaptUsedate, kaptMparea_60, kaptMparea_85, kaptMparea_135, kaptMparea_136, \
                                            privArea, bjdCode) \
                values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], \
                        row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18], row[19]))

conn.commit()
conn.close()

In [70]:
"""
import pymysql
import sqlalchemy

import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

engine = dbconnect.db_engine("DEMO_DW")

# MySQL Connector using pymysql
pymysql.install_as_MySQLdb()
conn = engine.connect()

#변수 타입 설정
dtypesql_data = {'kaptCode' : sqlalchemy.types.CHAR(9),
                 'kaptName' : sqlalchemy.types.VARCHAR(50),
                 'kaptAddr' : sqlalchemy.types.VARCHAR(200),
                 'codeSaleNm' : sqlalchemy.types.VARCHAR(50),
                 'codeHeatNm' : sqlalchemy.types.VARCHAR(50),
                 'kaptTarea' : sqlalchemy.types.DECIMAL(22,4),
                 'kaptDongCnt' : sqlalchemy.types.INTEGER(),
                 'kaptdaCnt' : sqlalchemy.types.INTEGER(),
                 'codeAptNm' : sqlalchemy.types.VARCHAR(50), 
                 'hoCnt' : sqlalchemy.types.INTEGER(),
                 'codeMgrNm' : sqlalchemy.types.VARCHAR(50),
                 'codeHallNm' : sqlalchemy.types.VARCHAR(50),
                 'kaptUsedate' : sqlalchemy.types.CHAR(8),
                 'kaptMparea_60' : sqlalchemy.types.INTEGER(),
                 'kaptMparea_85' : sqlalchemy.types.INTEGER(),
                 'kaptMparea_135' : sqlalchemy.types.INTEGER(),
                 'kaptMparea_136' : sqlalchemy.types.INTEGER(),
                 'privArea' : sqlalchemy.types.DECIMAL(22,4),
                 'bjdCode' : sqlalchemy.types.CHAR(10)}

df_save_data.to_sql(name='molit_apt_daegu', con=engine, if_exists='replace', index=True, dtype=dtypesql_data) 

conn.commit()
conn.close()
"""

In [ ]:
"""
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_save_data.itertuples():
    sql = "insert into molit_apt_daegu (kaptCode, kaptName, kaptAddr, codeSaleNm, codeHeatNm, kaptTarea, \
                                        kaptDongCnt, kaptdaCnt, codeAptNm, hoCnt, codeMgrNm, codeHallNm,  \
                                        kaptUsedate, kaptMparea_60, kaptMparea_85, kaptMparea_135, kaptMparea_136, \
                                        privArea, bjdCode) \
            values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], \
                      row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18], row[19]))

conn.commit()
conn.close()
"""